In [ ]:
import json
import matplotlib.pyplot as plt

In [ ]:
with open("../logs/experiment_bee_2_nodes.json", "r") as f:
    data = json.load(f)

train_epochs = [d["epoch"] for d in data if d["mode"] == "train"]
train_acc = [d["acc"] for d in data if d["mode"] == "train"]

test_epochs = [d["epoch"] for d in data if d["mode"] == "test"]
test_acc = [d["acc"] for d in data if d["mode"] == "test"]

plt.figure(figsize=(10, 5))
plt.plot(train_epochs, train_acc, label="Train Accuracy", marker='o')
plt.plot(test_epochs, test_acc, label="Test Accuracy", marker='x')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("[Bee] Accuracy over epochs when graph only has 2 nodes")
plt.legend()
plt.tight_layout()
plt.show()

In [ ]:
with open("../logs/experiment_human_2_nodes.json", "r") as f:
    data = json.load(f)

train_epochs = [d["epoch"] for d in data if d["mode"] == "train"]
train_acc = [d["acc"] for d in data if d["mode"] == "train"]

test_epochs = [d["epoch"] for d in data if d["mode"] == "test"]
test_acc = [d["acc"] for d in data if d["mode"] == "test"]

plt.figure(figsize=(10, 5))
plt.plot(train_epochs, train_acc, label="Train Accuracy", marker='o')
plt.plot(test_epochs, test_acc, label="Test Accuracy", marker='x')
plt.xlabel("Epoch")
plt.ylabel("Accuracy")
plt.title("[Human] Accuracy over epochs when graph only has 2 nodes")
plt.legend()
plt.tight_layout()
plt.show()